# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup

import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import json
import plotly.express as px

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mandal,25.4500,74.5667,13.57,63,0,0.46,IN,1674530914
1,1,udachnyy,66.4167,112.4000,-30.31,74,100,1.62,RU,1674530915
2,2,tuatapere,-46.1333,167.6833,17.73,47,7,3.89,NZ,1674530916
3,3,busselton,-33.6500,115.3333,28.76,41,21,5.03,AU,1674530916
4,4,gurlan,41.8447,60.3919,-12.96,35,7,2.51,UZ,1674530918
5,5,kpandu,6.9954,0.2931,25.51,78,60,1.93,GH,1674530918
6,6,punta arenas,-53.1500,-70.9167,10.06,57,0,7.72,CL,1674530919
7,7,qaanaaq,77.4840,-69.3632,-24.42,70,94,3.22,GL,1674530920
8,8,puerto ayora,-0.7393,-90.3518,23.64,98,89,1.34,EC,1674530921
9,9,port elizabeth,-33.9180,25.5701,21.84,86,0,11.62,ZA,1674530921


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
df = px.data.gapminder()
  
plot = px.scatter_geo(city_data_df,
                      lat= "Lat", lon= "Lng",
                      size="Humidity", size_max =7,
                      color = "City")
plot.show()


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
## A max temperature lower than 27 degrees but higher than 21

narrow_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) &(city_data_df["Max Temp"] > 21)]

## #Wind speed less than 4.5 m/s

narrow_df = narrow_df.loc[(narrow_df["Wind Speed"] < 4.5)]

## Zero cloudiness

narrow_df = narrow_df.loc[(narrow_df["Cloudiness"] == 0)]

# Drop any rows with null values

narrow_df = narrow_df.dropna()

# Display sample data
narrow_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
43,43,bahia blanca,-38.7196,-62.2724,26.27,43,0,0.89,AR,1674530954
123,123,mecca,21.4267,39.8261,21.35,75,0,1.22,SA,1674531001
129,129,gaoua,10.2992,-3.2508,25.23,22,0,1.00,BF,1674531049
130,130,puerto escondido,15.8500,-97.0667,25.46,73,0,0.00,MX,1674531050
200,200,maun,-19.9833,23.4167,21.98,78,0,1.54,BW,1674531111
246,246,pucallpa,-8.3791,-74.5539,25.02,78,0,3.09,PE,1674531158
335,335,boa vista,2.8197,-60.6733,24.99,73,0,2.57,BR,1674531245
340,340,presidencia roque saenz pena,-26.7852,-60.4388,24.86,43,0,2.83,AR,1674531250
379,379,cayenne,4.9333,-52.3333,25.02,83,0,1.54,GF,1674531296
454,454,acapulco,16.8634,-99.8901,25.90,73,0,2.57,MX,1674531280


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(city_data_df, columns = ["City", "Country", "Lat", "Lng", "Humidity"])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,mandal,IN,25.4500,74.5667,63,
1,udachnyy,RU,66.4167,112.4000,74,
2,tuatapere,NZ,-46.1333,167.6833,47,
3,busselton,AU,-33.6500,115.3333,41,
4,gurlan,UZ,41.8447,60.3919,35,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"

params = {
    "categories": categories,
    # "limit": limit,
    "apiKey": geoapify_key}
    

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
   
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
        
       
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mandal - nearest hotel: No hotel found
udachnyy - nearest hotel: Вилюй
tuatapere - nearest hotel: No hotel found
busselton - nearest hotel: Broadwater Beach Resort
gurlan - nearest hotel: No hotel found
kpandu - nearest hotel: Catherine’s Lodge
punta arenas - nearest hotel: Hotel Lacolet
qaanaaq - nearest hotel: Hotel Qaanaaq
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
port elizabeth - nearest hotel: Waterford Hotel
eureka - nearest hotel: Eureka Inn, Trademark Collection by Wyndham
port-cartier - nearest hotel: No hotel found
kodiak - nearest hotel: Shelikof Lodge
moundou - nearest hotel: فندق جمرا
castro - nearest hotel: Hotel Central Palace
mataura - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
andenes - nearest hotel: Andrikken hotell
georgetown - nearest hotel: Page 63 hostel
olafsvik - nearest hotel: North Star Hotel Snæfellsnes
richards bay - nearest hotel: Protea Hotel
acatic - nearest hotel: Motel Li

,City,Country,Lat,Lng,Humidity,Hotel Name
0,mandal,IN,25.4500,74.5667,63,No hotel found
1,udachnyy,RU,66.4167,112.4000,74,Вилюй
2,tuatapere,NZ,-46.1333,167.6833,47,No hotel found
3,busselton,AU,-33.6500,115.3333,41,Broadwater Beach Resort
4,gurlan,UZ,41.8447,60.3919,35,No hotel found
...,...,...,...,...,...,...
536,mancio lima,BR,-7.6142,-72.8958,94,No hotel found
537,thaba-tseka,LS,-29.5220,28.6084,56,Mohale oa Masitse Lodge
538,zeya,RU,53.7500,127.2667,72,Серебряный створ
539,lincoln,US,40.8000,-96.6670,89,New Victorian Suites Lincoln


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
df = px.data.gapminder()
  
plot = px.scatter_geo(hotel_df,
                      lat= "Lat", lon= "Lng",
                      size="Humidity", size_max =7, hover_data = ["Hotel Name", "Country"],
                      color = "City", scope='world')
plot.show()

# Display the map
